In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:


import pandas as pd

with open('/content/drive/MyDrive/ML IE506/ml-100k/ml-100k/u.data', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

# Process the lines to create a list of tuples
data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])
    # Skip the timestamp (parts[3])
    data.append((user_id, item_id, rating))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame

df['movieId'].max()
df['movieId'].min()

temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [3]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]

print(userItem)
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
(944, 1683)


In [4]:
R = userItem[1:]
print(R)
R = R[:, 1:]
print(R)

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

[[0. 5. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [5]:
A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)



In [11]:
import matplotlib.pyplot as plt
import math



def runALS(A, R, n_factors, n_iterations, lambda_):

    print ("Initiating ")
    lambda_ = 0.1; n_factors = 3; n, m = A.shape ; n_iterations=5
    Users = 5 * np.random.rand(n, n_factors)
    Items = 5 * np.random.rand(n_factors, m)


    def get_error(A, Users, Items, R):
        # This calculates the MSE of nonzero elements
        return math.sqrt(np.sum((R * (A - np.dot(Users, Items))) ** 2) / np.sum(R))

    RMSE_List = []

    print ("Starting Iterations")
    for iter in range(n_iterations):
        for i, Ri in enumerate(R):
            Users[i] = np.linalg.solve(np.dot(Items, np.dot(np.diag(Ri), Items.T)) + lambda_ * np.eye(n_factors),
                                       np.dot(Items, np.dot(np.diag(Ri), A[i].T))).T
        print ("Error after solving for User Matrix:", get_error(A, Users, Items, R))

        for j, Rj in enumerate(R.T):
            Items[:,j] = np.linalg.solve(np.dot(Users.T, np.dot(np.diag(Rj), Users)) + lambda_ * np.eye(n_factors),
                                     np.dot(Users.T, np.dot(np.diag(Rj), A[:, j])))
        print ("Error after solving for Item Matrix:", get_error(A, Users, Items, R))

        RMSE_List.append(get_error(A, Users, Items, R))
        print ('%sth iteration is complete...' % iter)


    return RMSE_List.pop()




In [13]:
Rmse = runALS(R,A ,3,  5, 0.06)

Initiating 
Starting Iterations
Error after solving for User Matrix: 1.5165745561323372
Error after solving for Item Matrix: 0.9218603023001812
0th iteration is complete...
Error after solving for User Matrix: 0.8868933374147253
Error after solving for Item Matrix: 0.8731209195402303
1th iteration is complete...
Error after solving for User Matrix: 0.8636836499826951
Error after solving for Item Matrix: 0.8573561639786084
2th iteration is complete...
Error after solving for User Matrix: 0.8518504768384344
Error after solving for Item Matrix: 0.8484063722916344
3th iteration is complete...
Error after solving for User Matrix: 0.8453591989339289
Error after solving for Item Matrix: 0.8435795270060242
4th iteration is complete...


In [15]:
ep = [0.5, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
rmse_CALS = []

for ep in ep:
    rmse_CALS.append(Rmse)

print('The RMSE for different epsilons are', rmse_CALS)

The RMSE for different epsilons are [0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242, 0.8435795270060242]
